### **Es muy común que nuestros DataFrames presenten datos faltantes, antes de empezar a procesar nuestros DataFrames veamos un poco en qué consisten los objetos NaN (Not a Number).**

Importemos las librerias Pandas y Numpy para esto:

In [27]:
import numpy as np
import pandas as pd

Un número que no está definido usualmente se representa con el siguiente objeto:


In [28]:
np.nan

nan

¡Este objeto tiene propiedades matemáticas! Al sumar un número, obtenemos como
respuesta el mismo NaN.

In [29]:
np.nan + 0

nan

In [30]:
np.nan > 0

False

La versión 1.0 de pandas incluye un nuevo objeto NA, que es mucho más
general pues, ademas de interactuar con números, tambien puede hacerlo con
cadenas de texto u otras variables como las de tipo booleano. Si quieres que
esta nueva definición este incluida entre tus cálculos usa:

In [31]:
 pd.options.mode.use_inf_as_na = True

Al sumar NA a una cadena de texto, obtengo el mismo NA:

In [32]:
pd.NA +'Hola mundo'

<NA>

In [33]:
pd.NA | False

<NA>

***A continuación, vamos a crear un DataFrame***



In [34]:
df = pd.DataFrame(np.arange(0, 15).reshape(5, 3), columns=['a', 'b', 'c'])
df

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11
4,12,13,14


Y vamos a añadir algunas variables no definidas:

In [35]:
df['d'] = np.nan
df['e'] = np.arange(15, 20)
df.loc[5,:] = pd.NA
df.loc[4,'a'] = pd.NA
df.loc[0,'d'] = 1
df.loc[5,'d'] = 10
df

,a,b,c,d,e
0,0,1,2,1,15
1,3,4,5,NaN,16
2,6,7,8,NaN,17
3,9,10,11,NaN,18
4,<NA>,13,14,NaN,19
5,<NA>,<NA>,<NA>,10,<NA>


Para reconocer cuando un objeto es nulo simplemente usamos:



In [36]:
df.isnull()

,a,b,c,d,e
0,False,False,False,False,False
1,False,False,False,True,False
2,False,False,False,True,False
3,False,False,False,True,False
4,True,False,False,True,False
5,True,True,True,False,True


En dónde todas nuestras variables no definidas fueron marcadas con TRUE,
df.isna() tambien cumple esta función.

Conocer el número de variables nulas por columna puede hacerse juntando el
comando anterior con la funcion de suma:

In [37]:
df.isnull().sum()

a    2
b    1
c    1
d    4
e    1
dtype: int64

dtype: int64

Si lo que nos interesa es conocer el número de filas con elementos nulos, basta
con usar axis=1:

In [38]:
df.notnull().sum(axis=1)

0    5
1    4
2    4
3    4
4    3
5    1
dtype: int64


dtype: int64

O todos los elementos nulos de nuestro DataFrame:

In [39]:
df.size-df.isnull().sum().sum()

21

Reconocer estos elementos nos puede ayudar a filtrar en nuestro DataFrame, en
este caso, me gustaría filtrar por las variables no nulas de la columna ‘a’:

In [40]:
df[df['a'].notnull()]

,a,b,c,d,e
0,0,1,2,1,15
1,3,4,5,NaN,16
2,6,7,8,NaN,17
3,9,10,11,NaN,18


dropna es perfecto para elimnar rapidamente las filas con registros faltantes:

In [41]:
df.dropna()

,a,b,c,d,e
0,0,1,2,1,15


In [42]:
df[['a']].dropna()

,a
0,0
1,3
2,6
3,9


Ya que hemos visto cómo funcionan las variable nulas, veamos cómo lidiar con
ellas. Usando la función fillna podremos reemplazarlas por el valor que
querramos, en este caso 0.

In [43]:
df.fillna(0)

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,0,16.0
2,6.0,7.0,8.0,0,17.0
3,9.0,10.0,11.0,0,18.0
4,0.0,13.0,14.0,0,19.0
5,0.0,0.0,0.0,10,0.0


Si quisieramos remplazar con el valor siguiente usamos
method="ffill", remplazo a los ceros(valores nulos) con el numero anterior en la misma columna

In [44]:
df.fillna(method="ffill")

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,1,16.0
2,6.0,7.0,8.0,1,17.0
3,9.0,10.0,11.0,1,18.0
4,9.0,13.0,14.0,1,19.0
5,9.0,13.0,14.0,10,19.0


Si quisieramos remplazar con el valor previo usamos
method="bfill", remplazo a los ceros (valores nulos) con el numero siguiente en la misma columna

In [45]:
df.fillna(method="bfill")

,a,b,c,d,e
0,0,1,2,1,15
1,3,4,5,10,16
2,6,7,8,10,17
3,9,10,11,10,18
4,<NA>,13,14,10,19
5,<NA>,<NA>,<NA>,10,<NA>


El mismo ejercicio anterior se puede aplicar con las filas usando axis=1:

In [46]:
df.fillna(method="bfill",axis=1)

,a,b,c,d,e
0,0.0,1.0,2.0,1.0,15
1,3.0,4.0,5.0,16.0,16
2,6.0,7.0,8.0,17.0,17
3,9.0,10.0,11.0,18.0,18
4,13.0,13.0,14.0,19.0,19
5,10.0,10.0,10.0,10.0,<NA>


Podemos usar también una serie para reemplazar los valores de una columna en especifico, es importante que haya emparejamiento entre los índices:

In [47]:
fill = pd.Series([100, 101, 102])
fill

0    100
1    101
2    102
dtype: int64

In [48]:
df['d'] = df['d'].fillna(fill)
df['d'] 

0      1.0
1    101.0
2    102.0
3      NaN
4      NaN
5     10.0
Name: d, dtype: float64

In [49]:
df

,a,b,c,d,e
0,0,1,2,1.0,15
1,3,4,5,101.0,16
2,6,7,8,102.0,17
3,9,10,11,NaN,18
4,<NA>,13,14,NaN,19
5,<NA>,<NA>,<NA>,10.0,<NA>


Una de las formas más usadas para reemplazar datos es usar el promedio de las columnas, esto se hace con la función mean. O si se quiere un mejor estimador, usamos median.

In [50]:
df.fillna(df.median())


,a,b,c,d,e
0,0.0,1.0,2.0,1.0,15.0
1,3.0,4.0,5.0,101.0,16.0
2,6.0,7.0,8.0,102.0,17.0
3,9.0,10.0,11.0,55.5,18.0
4,4.5,13.0,14.0,55.5,19.0
5,4.5,7.0,8.0,10.0,17.0


Por último, Pandas también puede interporlar los valores faltanes calculando el valor que puede haber existido en el medio.

df_d = pd.concat([df[['d']], df[['d']].interpolate()],axis=1)
df_d.columns = ['d_antes','d_interpolado']
df_d